# codex_prompting.ipynb

Author: Gabriel Grand (grandg@mit.edu) | Date: 01-30-23

Code for prompting codex to generate conditioning statements for physical language tasks.

In [47]:
import openai
import os
import pandas as pd
import re
import datetime
import pickle
import json

from tqdm import tqdm

In [35]:
# Configure OpenAI API

openai.api_key = os.environ["OPENAI_API_KEY"]

ENGINE = "code-davinci-002"
MAX_TOKENS = 512
N_SAMPLES = 1
TEMPERATURE = 0.0
STOP = "/**"

In [36]:
RANDOM_SEED = 123

DIR_DATA = "data"
DIR_EXPERIMENTS = "../experiments"
DIR_CODEX = "codex"
DIR_COMPLETIONS = "completions"
DIR_PROMPTS = "prompts"

FILE_GENERATIVE_MODEL = os.path.join(DIR_DATA, "generative_model.js")
# FILE_STIMULI = os.path.join(DIR_DATA, "phys_lang_examples.csv")
FILE_STIMULI = os.path.join(DIR_DATA, "stimuli_v1.csv")

In [37]:
with open(FILE_GENERATIVE_MODEL, "r") as f:
    generative_model_text = f.read()

In [38]:
print(f"{generative_model_text}")

/**
 * WebPPL generative model of a blockworld.
 */
var makeBlockWorld = function () {

    //// Distributions and parameters ////

    var truncGeom = function (p, m, n) {
        if (m > n) {
            return uniformDraw(_.range(1, n + 1));
        } else {
            return flip(p) ? truncGeom(p, m + 1, n) : m;
        }
    }

    var dim = 10;
    var tableSize = 100;
    var worldWidth = 600;
    var worldHeight = 500;
    var color = function () { return flip() ? 'red' : 'yellow' };
    var monoColor = flip(1.0);
    var stackHeight = function () { return truncGeom(0.7, 1, 8) };
    var numStacks = truncGeom(0.5, 1, 8);
    var xpositions = _.range(worldWidth / 2 - tableSize, worldWidth / 2 + tableSize + 20, 20);

    //// Object definitions ////

    var ground = {
        shape: 'rect',
        static: true,
        dims: [100000 * worldWidth, 10],
        x: worldWidth / 2,
        y: worldHeight
    }

    var table = {
        shape: 'rect',
        static: false,
      

In [39]:
df = pd.read_csv(FILE_STIMULI, index_col="task_id", keep_default_na=False)
df

,language_full,number,location,quant_or_neg,vague_adj,language_phrase_1,language_phrase_2,language_phrase_3,language_phrase_4,code_full,code_phrase_1,code_phrase_2,code_phrase_3,code_phrase_4,complexity,example_answer
task_id,,,,,,,,,,,,,,,,
1,"There are four stacks of red blocks, and there...",1,,,,There are four stacks of red blocks.,There is one stack of yellow blocks.,,,"condition(filter(isRed, world.stacks).length =...","condition(filter(isRed, world.stacks).length =...","condition(filter(isYellow, world.stacks).lengt...",,,low,2
2,"There are two stacks of yellow blocks, and the...",1,,,,There are two stacks of yellow blocks.,There is one stack of red blocks.,,,"condition(filter(isYellow, world.stacks).lengt...","condition(filter(isYellow, world.stacks).lengt...","condition(filter(isRed, world.stacks).length =...",,,low,5
3,"There are two stacks of red blocks, and there ...",1,,,,There are two stacks of red blocks.,There are two stacks of yellow blocks.,,,"condition(filter(isRed, world.stacks).length =...","condition(filter(isRed, world.stacks).length =...","condition(filter(isYellow, world.stacks).lengt...",,,low,4
4,There are yellow blocks on the left side of th...,,1,,,There are yellow blocks on the left side of th...,There are red blocks on the right edge of the ...,,,"condition(filter(isOnLeft, filter(isYellow, wo...","condition(filter(isOnLeft, filter(isYellow, wo...","condition(filter(isOnEdge, filter(isOnRight, f...",,,low,3
5,There are red blocks on the left edge of the t...,,1,,,There are red blocks on the left edge of the t...,There are yellow blocks on the right edge of t...,,,"condition(filter(isOnEdge, filter(isOnLeft, fi...","condition(filter(isOnEdge, filter(isOnLeft, fi...","condition(filter(isOnEdge, filter(isOnRight, f...",,,low,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,There is at least one tall stack of red blocks...,1,1,1,1,There is at least one tall stack of red blocks...,There are two stacks of yellow blocks on the r...,There are no blocks on the left side.,,"condition(filter(isOnMiddle, filter(isTall, fi...","condition(filter(isOnMiddle, filter(isTall, fi...","condition(filter(isOnRight, filter(isYellow, w...","condition(filter(isOnLeft, world.blocks).lengt...",,high,5
61,There are at least two tall stacks of yellow b...,1,1,1,1,There are at least two tall stacks of yellow b...,There is at most one tall stack of red blocks ...,,,"condition(filter(isOnLeft, filter(isTall, filt...","condition(filter(isOnLeft, filter(isTall, filt...","condition(filter(isOnRight, filter(isTall, fil...",,,high,6
62,There is one tall stack of yellow blocks on th...,1,1,2,1,There is one tall stack of yellow blocks on th...,There are no red blocks on the right edge.,,,"condition(filter(isOnEdge, filter(isOnLeft, fi...","condition(filter(isOnEdge, filter(isOnLeft, fi...","condition(filter(isOnEdge, filter(isOnRight, f...",,,high,6


In [40]:
n_phrases_langauge = len(list(filter(lambda col_name: "language_phrase_" in col_name, df.columns)))
n_phrases_code = len(list(filter(lambda col_name: "code_phrase_" in col_name, df.columns)))
assert(n_phrases_langauge == n_phrases_code)
N_PHRASES = n_phrases_langauge
print(N_PHRASES)

4


In [48]:
TEMPLATE_EXAMPLE_HEADER = """
/**
 * Example:
 * {language_full}
 */
"""

def construct_prompt(df, task_id, global_header, n_prompt_examples: int = None):
    prompt = global_header + "\n"
    
    # hold one out
    df_examples = df.drop(df.loc[[task_id]].index)
    
    if n_prompt_examples:
        df_examples = df_examples.sample(n=n_prompt_examples, random_state=RANDOM_SEED)
    
    for _, row in df_examples.iterrows():
        example_header = TEMPLATE_EXAMPLE_HEADER.format(language_full=row["language_full"])
        prompt += example_header
        
        for i in range(1, N_PHRASES + 1):
            if row[f"language_phrase_{i}"]:
                prompt += "\n"
                prompt += "// " + row[f"language_phrase_{i}"] + "\n"
                prompt += row[f"code_phrase_{i}"] + "\n"
                
    prompt += TEMPLATE_EXAMPLE_HEADER.format(language_full=df.loc[[task_id], "language_full"].item())
        
    return prompt, df_examples.index.tolist()

prompt, task_ids = construct_prompt(df, 1, generative_model_text)

In [ ]:
print(f"{prompt}")

In [50]:
def query_codex(prompt):
    completion = openai.Completion.create(
        engine=ENGINE,
        prompt=prompt,
        temperature=TEMPERATURE,
        n=N_SAMPLES,
        stop=STOP,
        max_tokens=MAX_TOKENS,
        logprobs=None,
    )
    
    return completion

In [51]:
def extract_conditions(text):
    start, end = "condition", ";"
    return [start + x + end for x in re.findall(str(re.escape(start)) + "(.*)" + str(re.escape(end)), text)]

def extract_language(text):
    start, end = "// ", "\n"
    return re.findall(str(re.escape(start)) + "(.*)" + str(re.escape(end)), text)

def parse_choice(choice):
    if choice.finish_reason != "stop":
        print(f"WARNING: Completion choice {choice.index} encountered non-terminal finish reason: {choice.finish_reason}")

    data = {
        "choice_index": choice.index,
        "finish_reason": choice.finish_reason,
        "text": choice.text,
    }
    
    for i, (language, code) in enumerate(zip(extract_language(choice.text), extract_conditions(choice.text))):
        data.update({
            f"codex_language_phrase_{i+1}": language,
            f"codex_code_phrase_{i+1}": code,
        })
        
    return data

In [70]:
def run_experiment(df: pd.DataFrame, experiment_id: str = None, restore_ckpt: bool = False, n_prompt_examples: int = 10):
    experiment_id = experiment_id or datetime.datetime.now().strftime('run-%Y-%m-%d-%H-%M-%S')
    ckpt_dir = os.path.join(DIR_EXPERIMENTS, experiment_id, DIR_CODEX)
    os.makedirs(os.path.join(ckpt_dir, DIR_COMPLETIONS), exist_ok=True)
    os.makedirs(os.path.join(ckpt_dir, DIR_PROMPTS), exist_ok=True)

    completions = []
    # Query OpenAI for completions
    for task_id in tqdm(df.index):
        completion_pkl = os.path.join(ckpt_dir, DIR_COMPLETIONS, f"completion_task_{task_id:03d}.pkl")
        
        # Try loading completion from file
        if restore_ckpt is not None:
            if os.path.exists(completion_pkl):
                with open(completion_pkl, "rb") as f:
                    completions.append(pickle.load(f))
                continue
            else:
                print(f"Completion not found: {completion_pkl}")
            
        prompt_text, prompt_task_ids = construct_prompt(df, task_id, generative_model_text, n_prompt_examples=n_prompt_examples)
        completion = query_codex(prompt_text)
        completions.append(completion)

        with open(completion_pkl, "wb") as f:
            pickle.dump(completion, f)
            
        with open(os.path.join(ckpt_dir, DIR_PROMPTS, f"prompt_task_{task_id:03d}.json"), "w") as f:
            prompt_json = {
                "task_id": task_id,
                "prompt_task_ids": prompt_task_ids,
                "prompt_text": prompt_text,
            }
            json.dump(prompt_json, f)

    # Parse completions data
    results = []
    for task_id, completion in zip(df.index, completions):
        for choice in completion.choices:
            d = {"task_id": task_id}
            d.update(parse_choice(choice))
            results.append(d)

    results_json = {"results": results}
    
    df_results = pd.DataFrame(results_json["results"]).fillna('')
    df_results = df_results.set_index("task_id")
    
    df_results = df.join(df_results)
    df_results.to_csv(os.path.join(DIR_EXPERIMENTS, RUN_ID, DIR_CODEX, "results.csv"))
    
    return df_results

In [71]:
# df_results = run_experiment(df)

RUN_ID = "run-2023-01-30-12-55-36"
df_results = run_experiment(df, experiment_id=RUN_ID, restore_ckpt=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 5994.67it/s]


In [72]:
df_results

,language_full,number,location,quant_or_neg,vague_adj,language_phrase_1,language_phrase_2,language_phrase_3,language_phrase_4,code_full,...,finish_reason,text,codex_language_phrase_1,codex_code_phrase_1,codex_language_phrase_2,codex_code_phrase_2,codex_language_phrase_3,codex_code_phrase_3,codex_language_phrase_4,codex_code_phrase_4
task_id,,,,,,,,,,,,,,,,,,,,,
1,"There are four stacks of red blocks, and there...",1,,,,There are four stacks of red blocks.,There is one stack of yellow blocks.,,,"condition(filter(isRed, world.stacks).length =...",...,stop,\n// There are four stacks of red blocks.\ncon...,There are four stacks of red blocks.,"condition(filter(isRed, world.stacks).length =...",There is one stack of yellow blocks.,"condition(filter(isYellow, world.stacks).lengt...",,,,
2,"There are two stacks of yellow blocks, and the...",1,,,,There are two stacks of yellow blocks.,There is one stack of red blocks.,,,"condition(filter(isYellow, world.stacks).lengt...",...,stop,\n// There are two stacks of yellow blocks.\nc...,There are two stacks of yellow blocks.,"condition(filter(isYellow, world.stacks).lengt...",There is one stack of red blocks.,"condition(filter(isRed, world.stacks).length =...",,,,
3,"There are two stacks of red blocks, and there ...",1,,,,There are two stacks of red blocks.,There are two stacks of yellow blocks.,,,"condition(filter(isRed, world.stacks).length =...",...,stop,\n// There are two stacks of red blocks.\ncond...,There are two stacks of red blocks.,"condition(filter(isRed, world.stacks).length =...",There are two stacks of yellow blocks.,"condition(filter(isYellow, world.stacks).lengt...",,,,
4,There are yellow blocks on the left side of th...,,1,,,There are yellow blocks on the left side of th...,There are red blocks on the right edge of the ...,,,"condition(filter(isOnLeft, filter(isYellow, wo...",...,stop,\n// There are yellow blocks on the left side ...,There are yellow blocks on the left side of th...,"condition(filter(isOnLeft, filter(isYellow, wo...",There are red blocks on the right edge of the ...,"condition(filter(isOnRight, filter(isRed, worl...",,,,
5,There are red blocks on the left edge of the t...,,1,,,There are red blocks on the left edge of the t...,There are yellow blocks on the right edge of t...,,,"condition(filter(isOnEdge, filter(isOnLeft, fi...",...,stop,\n// There are red blocks on the left edge of ...,There are red blocks on the left edge of the t...,"condition(filter(isOnLeft, filter(isRed, world...",There are yellow blocks on the right edge of t...,"condition(filter(isOnRight, filter(isYellow, w...",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,There is at least one tall stack of red blocks...,1,1,1,1,There is at least one tall stack of red blocks...,There are two stacks of yellow blocks on the r...,There are no blocks on the left side.,,"condition(filter(isOnMiddle, filter(isTall, fi...",...,stop,\n// There is at least one tall stack of red b...,There is at least one tall stack of red blocks...,"condition(filter(isOnMiddle, filter(isTall, fi...",There are two stacks of yellow blocks on the r...,"condition(filter(isOnRight, filter(isYellow, w...",There are no blocks on the left side.,"condition(filter(isOnLeft, world.blocks).lengt...",,
61,There are at least two tall stacks of yellow b...,1,1,1,1,There are at least two tall stacks of yellow b...,There is at most one tall stack of red blocks ...,,,"condition(filter(isOnLeft, filter(isTall, filt...",...,stop,\n// There are at least two tall stacks of yel...,There are at least two tall stacks of yellow b...,"condition(filter(isOnLeft, filter(isTall, filt...",There is at most one tall stack of red blocks ...,"condition(filter(isOnRight, filter(isTall, fil...",,,,
62,There is one tall stack of yellow blocks on th...,1,1,2,1,There is one tall stack of yellow blocks on th...,There are no red blocks on the right edge.,,,"condition(filter(isOnEdge, filter(isOnLeft, fi...",...,stop,\n

In [73]:
for i in range(1, N_PHRASES + 1):
    df_nonempty_language = df_results[df_results[f"language_phrase_{i}"] != '']
    df_nonempty_code = df_results[df_results[f"code_phrase_{i}"] != '']
    
    agreement_language = (df_nonempty_language[f"language_phrase_{i}"] == df_nonempty_language[f"codex_language_phrase_{i}"]).sum() / len(df_nonempty_language)
    agreement_code = (df_nonempty_code[f"code_phrase_{i}"] == df_nonempty_code[f"codex_code_phrase_{i}"]).sum() / len(df_nonempty_code)
    print(f"language_phrase_{i}: {agreement_language:03f}")
    print(f"code_phrase_{i}: {agreement_code:03f}")

language_phrase_1: 0.953125
code_phrase_1: 0.750000
language_phrase_2: 0.883333
code_phrase_2: 0.766667
language_phrase_3: 1.000000
code_phrase_3: 0.866667
language_phrase_4: 1.000000
code_phrase_4: 0.833333


In [86]:
from IPython.display import display

with pd.option_context('display.max_colwidth', None):
    for i in range(1, N_PHRASES + 1):
        display(df_results[f"code_phrase_{i}"].compare(df_results[f"codex_code_phrase_{i}"], result_names=('truth', 'codex')))

,truth,codex
task_id,,
5,"condition(filter(isOnEdge, filter(isOnLeft, filter(isRed, world.blocks))).length > 0);","condition(filter(isOnLeft, filter(isRed, world.blocks)).length > 0);"
8,"condition(filter(isYellow, world.blocks).length > world.blocks.length/2);","condition(filter(isYellow, world.blocks).length > filter(isRed, world.blocks).length);"
9,"condition(filter(isRed, world.blocks).length >= 6);","condition(filter(isRed, world.blocks).length > 5);"
10,"condition(filter(isYellow, world.stacks).length > 2 && filter(isYellow, world.stacks).length <= 5);","condition(filter(isYellow, world.stacks).length > 1 && filter(isYellow, world.stacks).length <= 4);"
11,"condition(filter(isYellow, world.blocks).length == world.blocks.length/2);","condition(filter(isYellow, world.blocks).length == filter(isRed, world.blocks).length);"
13,"condition(filter(isYellow, world.blocks).length >= world.blocks.length/2);","condition(filter(isYellow, world.blocks).length >= world.blocks.length / 2);"
25,condition(world.blocks.length >= 6);,condition(world.blocks.length >= 10);
31,"condition(filter(isRed, world.stacks).length > 2 && filter(isRed, world.stacks).length <= 5);","condition(filter(isRed, world.stacks).length > 2);"
48,"condition(filter(isOnEdge, filter(isOnLeft, filter(isRed, world.stacks))).length == 1);","condition(filter(isOnEdge, filter(isRed, world.stacks)).length == 1);"


,truth,codex
task_id,,
4,"condition(filter(isOnEdge, filter(isOnRight, filter(isRed, world.blocks))).length > 0);","condition(filter(isOnRight, filter(isRed, world.blocks)).length > 0);"
5,"condition(filter(isOnEdge, filter(isOnRight, filter(isYellow, world.blocks))).length > 0);","condition(filter(isOnRight, filter(isYellow, world.blocks)).length > 0);"
9,"condition(filter(isYellow, world.blocks).length > 1 && filter(isYellow, world.blocks).length <= 4);","condition(filter(isYellow, world.blocks).length > 1 && filter(isYellow, world.blocks).length <= 5);"
10,"condition(filter(isRed, world.stacks).length <= 4);","condition(filter(isRed, world.stacks).length > 0 && filter(isRed, world.stacks).length <= 2);"
11,"condition(filter(isRed, world.blocks).length == world.blocks.length/2);",
25,"condition(filter(isYellow, world.blocks).length > world.blocks.length/2);","condition(filter(isYellow, world.blocks).length >= world.blocks.length / 2);"
46,"condition(filter(isYellow, filter(isOnEdge, world.blocks)).length <= filter(isOnEdge, world.blocks).length/2);","condition(filter(isOnEdge, filter(isYellow, world.blocks)).length <= filter(isOnEdge, world.blocks).length/2);"
48,"condition(filter(isNear(filter(isOnEdge, filter(isOnLeft, filter(isRed, world.stacks)))[0]), filter(isTall, filter(isYellow, world.stacks))).length == 2);","condition(filter(isNear(filter(isOnEdge, filter(isRed, world.stacks))[0]), filter(isTall, filter(isYellow, world.stacks))).length == 2);"
51,"condition(filter(isYellow, world.stacks).length > 0 && all(isShort, filter(isYellow, world.stacks)));","condition(all(isShort, filter(isYellow, world.stacks)));"


,truth,codex
task_id,,
27,"condition(filter(isRed, filter(isOnRight, world.blocks)).length > filter(isOnRight, world.blocks).length/2);","condition(filter(isOnRight, filter(isRed, world.blocks)).length > world.blocks.length/2);"
57,"condition(filter(isTall, filter(isRed, world.stacks)).length <= 4);","condition(filter(isTall, filter(isRed, world.stacks)).length < 4);"


,truth,codex
task_id,,
39,"condition(any(isShort, filter(isYellow, world.stacks)));","condition(filter(isTall, filter(isYellow, world.stacks)).length > 0);"


In [87]:
for task_id, row in df_results.iterrows():
    print("---------")
    print(f"task_id {task_id}: {row['language_full']}")
    print()
    
    for i in range(1, N_PHRASES + 1):
        if row[f"language_phrase_{i}"]:
            print(f"Language {i}")
            print(f"TRUTH: {row[f'language_phrase_{i}']}")
            print(f"CODEX: {row[f'codex_language_phrase_{i}']}")
            print()

            print(f"Code {i}")
            print(f"TRUTH: {row[f'code_phrase_{i}']}")
            print(f"CODEX: {row[f'codex_code_phrase_{i}']}")
            print()

---------
task_id 1: There are four stacks of red blocks, and there is one stack of yellow blocks.

Language 1
TRUTH: There are four stacks of red blocks.
CODEX: There are four stacks of red blocks.

Code 1
TRUTH: condition(filter(isRed, world.stacks).length == 4);
CODEX: condition(filter(isRed, world.stacks).length == 4);

Language 2
TRUTH: There is one stack of yellow blocks.
CODEX: There is one stack of yellow blocks.

Code 2
TRUTH: condition(filter(isYellow, world.stacks).length == 1);
CODEX: condition(filter(isYellow, world.stacks).length == 1);

---------
task_id 2: There are two stacks of yellow blocks, and there is one stack of red blocks.

Language 1
TRUTH: There are two stacks of yellow blocks.
CODEX: There are two stacks of yellow blocks.

Code 1
TRUTH: condition(filter(isYellow, world.stacks).length == 2);
CODEX: condition(filter(isYellow, world.stacks).length == 2);

Language 2
TRUTH: There is one stack of red blocks.
CODEX: There is one stack of red blocks.

Code 2
TRUTH: